# Single-cell RNA-seq imputation using DeepImpute

Here is a comprehensive tutorial to understand the functionnalities of DeepImpute.

In [ ]:
from deepimpute.multinet import MultiNet
import pandas as pd

# Load dataset using pandas
data = pd.read_csv('test.csv',index_col=0)
print('Working on {} cells and {} genes'.format(*data.shape))

## Create a DeepImpute multinet

In [ ]:
# Using default parameters
multinet = MultiNet() 

In [ ]:
# Using custom parameters
NN_params = {
        'learning_rate': 1e-4,
        'batch_size': 64,
        'max_epochs': 200,
        'ncores': 5,
        'sub_outputdim': 512,
        'architecture': [
            {"type": "dense", "activation": "relu", "neurons": 200},
            {"type": "dropout", "activation": "dropout", "rate": 0.3}]
    }

multinet = MultiNet(**NN_params)

## Fit the networks

In [ ]:
# Using all the data
multinet.fit(data,cell_subset=1,minVMR=0.5)

In [ ]:
# Using 80% of the data
multinet.fit(data,cell_subset=0.5)

In [ ]:
# Using 200 cells (randomly selected)
multinet.fit(data,cell_subset=200)

In [ ]:
# Custom fit
trainingData = data.iloc[100:250,:]
multinet.fit(trainingData)

## Imputation

The imputation can be done on any dataset as long as the gene labels are the same

In [ ]:
imputedData = multinet.predict(data)

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

limits = [0,100]

fig,ax = plt.subplots()

jitter = np.random.normal(0,1,data.size) # Add some jittering to better see the point density
ax.scatter(data.values.flatten()+jitter,imputedData.values.flatten(),s=2)
ax.plot(limits,limits,'r-.',linewidth=2)
ax.set_xlim(limits)
ax.set_ylim(limits)

plt.show()


## Scoring
Display training metrics (MSE and Pearson's correlation on the test data)

In [ ]:
multinet.test_metrics